In [10]:
import pandas as pd
import os
from tqdm import tqdm
from dotenv import load_dotenv
from zhipuai import ZhipuAI
load_dotenv()
client = ZhipuAI(api_key=os.getenv("ZHIPUAI_API_KEY"))

In [5]:
df_chunks = pd.read_csv("preprocessed/chunks.csv", sep="|", index_col=0)
df_chunks

,text,source,page
0,李产业发展现状\n一、中国李产业现状\n(一)栽培历史\n我国李栽培历史久远，原始居民就有采...,./李高效生产及绿色防控技术.pdf,0
1,李高效生产及绿色防控技术\n考证，“藉栽”指自根繁殖，即包括扦插、压条、分株等无性繁殖方\n...,./李高效生产及绿色防控技术.pdf,1
2,"第一章李产业发展现状\n年积温在4500~8000℃,最冷月平均气温0~15℃,是热带与温带...",./李高效生产及绿色防控技术.pdf,2
3,李高效生产及绿色防控技术\n(3)资源品种。本区主要栽培种为中国李，部分为欧洲李、杏李\n和...,./李高效生产及绿色防控技术.pdf,3
4,第一章李产业发展现状\n2绥棱红(北方1号)，它的母本和父本分别来自于小黄李和福摩萨，\n由...,./李高效生产及绿色防控技术.pdf,4
...,...,...,...
145,第五章李贮运保鲜与加工\n(5)预煮。预煮前用1.5%的食盐水护色，最后用清水淘洗1次。\n...,./李高效生产及绿色防控技术.pdf,145
146,李高效生产及绿色防控技术\n2.主要加工设施设备\n(1)清洗设备。鼓泡清洗机、毛刷清洗机、...,./李高效生产及绿色防控技术.pdf,146
147,第五章李贮运保鲜与加工\n4.产品质量要求\n(1)感官指标。具有该品种李加工后应有的正常颜...,./李高效生产及绿色防控技术.pdf,147
148,李高效生产及绿色防控技术\n(3)清洗。采用果蔬专用清洗设备清洗表面灰尘、腐叶、虫卵等\n污...,./李高效生产及绿色防控技术.pdf,148


In [4]:
df_qa = pd.read_csv("preprocessed/qa.csv", sep="|", index_col=0)
df_qa

,q,a,d
0,请问中国李产业的历史可以追溯到什么时候？,远在5000多年前,0
1,中国最早关于李的文献记载出现在哪部古籍中？,《诗经》,0
2,《广志》记载汉代上林苑收集了多少个李品种？,15个,0
3,《尔雅》中记录了哪些李品种？,椄虑李、无实李、赤李,0
4,《山海经》中提到的灵山有哪些果树？,桃、李、梅、杏,0
...,...,...,...
838,李高效生产及绿色防控技术的具体步骤是怎样的？,李高效生产及绿色防控技术的步骤包括清洗、破碎打浆、加热浓缩、洗瓶、装罐及密封和杀菌冷却。,148
839,在李果酱的生产过程中，原浆是如何处理的？,在李果酱的生产过程中，原浆是通过破碎打浆机将李打成原浆，并在打浆过程中添加0.2%~0.5%...,148
840,李果酱在装罐封口时的温度要求是多少？,李果酱在装罐封口时的温度要求在80℃以上，以防止微生物污染。,148
841,李果酱的产品质量要求有哪些？,李果酱的产品质量要求包括感官指标、理化指标、污染物限量和微生物限量。感官指标要求具有该产品应...,148


In [12]:
SYSTEM_PROMPT_EVAL_RAG = """根据给定上下文，正确地问答问题，只需要输出答案，不需要输出解答过程。"""
SYSTEM_PROMPT_EVAL_VANILLA = """正确地问答问题，只需要输出答案，不需要输出解答过程。"""
def llm(text: str):
  res = client.chat.completions.create(
    model="glm-3-turbo",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPT_EVAL_VANILLA},
      {"role": "user", "content": text},
    ]
  )
  return res.choices[0].message

In [21]:
with open("eval/glm-3-turbo-vanilla.csv", "a") as w:
    for i, row in tqdm(df_qa.iterrows()):
        if i > 837:
            q = row["q"]
            a = row["a"]
            content = llm(f"问题: {q}").content
            w.write(f"{q}|{a}|{content}\n")
            w.flush()

0it [00:00, ?it/s]

843it [00:18, 45.20it/s] 


In [32]:
df_glm_3_turbo_vanilla = pd.read_csv("eval/glm-3-turbo-vanilla.csv", sep="|", names=["q", "hyp", "ref"]).dropna().reset_index(drop=True)
df_glm_3_turbo_vanilla

,q,hyp,ref
0,请问中国李产业的历史可以追溯到什么时候？,远在5000多年前,答案：不确定，需要更多信息来明确“李产业”的具体含义。如果指的是李姓的历史，李姓是中国的一个...
1,中国最早关于李的文献记载出现在哪部古籍中？,《诗经》,答：《史记》。
2,《广志》记载汉代上林苑收集了多少个李品种？,15个,答案: 千余种。
3,《尔雅》中记录了哪些李品种？,椄虑李、无实李、赤李,答案: 《尔雅》中没有明确记录李品种。
4,《山海经》中提到的灵山有哪些果树？,桃、李、梅、杏,答案: 《山海经》中没有提到灵山有特定的果树。
...,...,...,...
834,李高效生产及绿色防控技术的具体步骤是怎样的？,李高效生产及绿色防控技术的步骤包括清洗、破碎打浆、加热浓缩、洗瓶、装罐及密封和杀菌冷却。,整形修剪是李高效生产和绿色防控技术中的一个重要步骤。它主要包括以下几个方面：
835,在李果酱的生产过程中，原浆是如何处理的？,在李果酱的生产过程中，原浆是通过破碎打浆机将李打成原浆，并在打浆过程中添加0.2%~0.5%...,参考信息中没有提供关于李果酱生产过程中原浆处理的步骤。请提供更多信息以便我能给出准确的答案。
836,李果酱在装罐封口时的温度要求是多少？,李果酱在装罐封口时的温度要求在80℃以上，以防止微生物污染。,答案: 请提供具体的产品名称和相关的生产标准，以便我能给出准确的温度要求。
837,李果酱的产品质量要求有哪些？,李果酱的产品质量要求包括感官指标、理化指标、污染物限量和微生物限量。感官指标要求具有该产品应...,答案: 李果酱的产品质量要求通常包括以下几点：


In [25]:
SYSTEM_PROMPT_JUDGE = """你是一个问答题评判专家，你的任务是评判模型输出是否和参考答案一致。

如果完全一致，则输出 1
如果部分匹配，则输出 0.5
如果意思冲突，则输出 0

注意用词不一样但意思完全一样的情况也输出 1。
只需要输出结果，不需要输出分析过程。
"""
def judge(text: str):
  res = client.chat.completions.create(
    model="glm-3-turbo",
    messages=[
      {"role": "system", "content": SYSTEM_PROMPT_JUDGE},
      {"role": "user", "content": text},
    ]
  )
  return res.choices[0].message

In [30]:
i = 300
q = df_glm_3_turbo_vanilla.iloc[i]["q"]
hyp = df_glm_3_turbo_vanilla.iloc[i]["hyp"]
ref = df_glm_3_turbo_vanilla.iloc[i]["ref"]
print(q)
judge(f"模型输出: {hyp}\n参考答案{ref}")

整形修剪如何减少李树的病虫害发生


CompletionMessage(content='0.5', role='assistant', tool_calls=None)

In [31]:
with open("eval/glm-3-turbo-vanilla-eval.csv", "a") as w:
    for i, row in tqdm(df_glm_3_turbo_vanilla.iterrows()):
        if i >= 0:
            hyp = row["hyp"]
            ref = row["ref"]
            content = judge(f"模型输出: {hyp}\n参考答案{ref}").content
            w.write(f"{content}\n")
            w.flush()

839it [09:54,  1.41it/s]


In [43]:
df_glm_3_turbo_vanilla_eval = pd.read_csv("eval/glm-3-turbo-vanilla-eval.csv", names=["score"]).dropna()
df_glm_3_turbo_vanilla_eval["score"].sum() / len(df_glm_3_turbo_vanilla_eval)

0.23158522050059596